### Install Library

In [3]:
!pip install jupyterlab_code_formatter black isort ultralytics opencv-python easyocr pandas pillow matplotlib gTTS playsound==1.2.2

### Train Model

https://universe.roboflow.com/naruesorn/thai-license-plate-j6y9l/dataset/1#

```
open /opt/homebrew/runs/detect/thai_plate_v1_fast4/weights/
```

In [9]:
from ultralytics import YOLO

# Base Model
model = YOLO("yolov8n.pt")

results = model.train(
    data="./plate/data.yaml",
    epochs=50,  # จำนวนรอบในการดููรูปซ้ำๆ
    imgsz=640,  # Size Image
    batch=16,  # จำนวนรูปที่ดูพร้อมกันใน 1ครั้ง
    name="thai_plate_v1_fast",  # Name Dataset
    device="mps",  # GPU On Mac
)

print("✅ Success Train!")

New https://pypi.org/project/ultralytics/8.3.232 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.13.5 torch-2.9.1 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./plate/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=thai_plate_v1_fast4, nbs=64, nms=False, opset=None, optimize=False, 

### Webcam Check 4 Round

In [1]:
import csv
import os
import re
from datetime import datetime

import cv2
import easyocr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO

MODEL_PATH = "./Train/thai_plate_v1_fast4/weights/best.pt"
FONT_PATH = "/System/Library/Fonts/Supplemental/Ayuthaya.ttf"

CONFIDENCE = 0.15
OCR_INTERVAL = 15
SAME_PLATE_COUNT = 4
CLEAR_SCREEN = 45
PADDING = 15

last_text = None
last_img = None
pending = None
empty_frames = 0
count = 0
frame_counter = 0

Color_Box = (203, 192, 255)
Color_Bar = (180, 158, 255)
Color_Text = (255, 255, 255)
Color_Verify = (0, 165, 255)


os.makedirs("saved_plates", exist_ok=True)
if not os.path.exists("parking_log.csv"):
    with open("parking_log.csv", "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["Timestamp", "Plate", "Image"])

print("⏳ Loading AI...")
model = YOLO(MODEL_PATH)
reader = easyocr.Reader(["th"], gpu=True)
print("✅ Ready!")


def draw_text(img, text, pos, color=(255, 255, 255), size=30):

    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)

    font = (
        ImageFont.truetype(FONT_PATH, size * 2)
        if os.path.exists(FONT_PATH)
        else ImageFont.load_default()
    )

    draw.text((pos[0] + 2, pos[1] + 2), text, font=font, fill=(100, 100, 100))

    draw.text(pos, text, font=font, fill=color)

    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)


def clean_text(text):
    return re.sub(r"[^ก-๙0-9]", "", text)


def correct_text(text):
    m = re.match(r"^(\d)(.)(.)(\d+)$", text)
    if m:

        p, c1, c2, s = m.groups()
        c1 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c1, c1)
        c2 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c2, c2)
        return f"{p}{c1}{c2}{s}"
    return text


def process_plate(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    enhanced = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)

    res = reader.readtext(
        enhanced,
        detail=1,
        mag_ratio=2.0,
        allowlist="0123456789กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ",
    )

    full_text = "".join([clean_text(t) for _, t, _ in res])

    m = re.match(r"^([0-9ก-๙]+\d)(?![0-9])", full_text)

    if not m or len(m.group(1)) < 3:
        return None

    return correct_text(m.group(1))


def save_log(text, img):
    global last_text, last_img, pending, count
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"saved_plates/{text}_{ts}.jpg"
    cv2.imwrite(fname, img)
    with open("parking_log.csv", "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(
            [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), text, fname]
        )
    print(f"💾 Saved: {text}")

    last_text = text
    last_img = img
    pending = None
    count = 0
    return fname


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    detected = False
    h, w = frame.shape[:2]

    for r in model(frame, conf=CONFIDENCE, verbose=False, device="mps"):
        for box in r.boxes:
            detected = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            x1, y1 = max(0, x1 - PADDING), max(0, y1 - PADDING)
            x2, y2 = min(w, x2 + PADDING), min(h, y2 + PADDING)

            color = Color_Verify if count > 0 else Color_Box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

            if not last_text and frame_counter % OCR_INTERVAL == 0:

                plate_text = process_plate(frame[y1:y2, x1:x2])
                if plate_text:

                    if plate_text == pending:
                        count += 1
                    else:

                        pending = plate_text
                        count = 1

                    print(f"Checking: {plate_text} {count}/{SAME_PLATE_COUNT})")

                    if count >= SAME_PLATE_COUNT:
                        save_log(plate_text, frame[y1:y2, x1:x2])

    if not detected and not pending:
        empty_frames += 1
    else:
        empty_frames = 0

    if empty_frames > CLEAR_SCREEN:
        if last_text:
            print("🚗 Clear")

        last_text = None
        last_img = None
        pending = None
        count = 0
        empty_frames = 0

    msg = (
        f"บันทึกสำเร็จ: {last_text}"
        if last_text
        else (
            f"กำลังยืนยัน {count}/{SAME_PLATE_COUNT}: {pending}"
            if count > 0
            else (f"กำลังตรวจจับป้าย..." if detected else f"กำลังรอรถ...")
        )
    )

    cv2.rectangle(frame, (0, 0), (1280, 100), Color_Bar, -1)
    frame = draw_text(frame, msg, (10, 10), Color_Text, 30)

    if last_img is not None:
        try:

            thumbnail_height, thumbnail_width = 80, int(
                80 * (last_img.shape[1] / last_img.shape[0])
            )

            frame[
                10 : 10 + thumbnail_height, 1280 - thumbnail_width - 20 : 1280 - 20
            ] = cv2.resize(last_img, (thumbnail_width, thumbnail_height))
        except:
            pass

    cv2.imshow("Smart Parking", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

⏳ Loading AI...
✅ Ready!


[ WARN:0@3.929] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Checking: กจ3046 1/4)
Checking: กจ3046 2/4)
Checking: กจ3046 3/4)
Checking: กจ3046 4/4)
💾 Saved: กจ3046
🚗 Clear


In [ ]:
import csv  # นำเข้าไลบรารีสำหรับจัดการไฟล์ CSV เพื่อบันทึกประวัติการเข้า-ออก
import os  # นำเข้าไลบรารีสำหรับจัดการระบบปฏิบัติการ (เช่น การสร้างโฟลเดอร์)
import re  # นำเข้าไลบรารี Regular Expression (Regex) สำหรับการจับรูปแบบข้อความ
from datetime import datetime  # นำเข้าโมดูลสำหรับจัดการวันที่และเวลาปัจจุบัน

import cv2  # นำเข้า OpenCV ไลบรารีหลักสำหรับการประมวลผลภาพและวิดีโอ
import easyocr  # นำเข้า EasyOCR ไลบรารี AI สำหรับอ่านตัวอักษร (OCR)
import numpy as np  # นำเข้า NumPy สำหรับการจัดการข้อมูลภาพในรูปแบบ Array
from PIL import (  # นำเข้า Pillow ใช้สำหรับวาดข้อความภาษาไทยบนภาพ
    Image,
    ImageDraw,
    ImageFont,
)
from ultralytics import (
    YOLO,  # นำเข้า YOLO ไลบรารี AI สำหรับตรวจจับวัตถุ (Object Detection)
)

# --- CONFIGURATION & CONSTANTS ---
MODEL_PATH = "./Train/thai_plate_v1_fast4/weights/best.pt"  # กำหนดพาธของไฟล์โมเดล YOLO ที่ใช้ตรวจจับป้ายทะเบียน
FONT_PATH = "/System/Library/Fonts/Supplemental/Ayuthaya.ttf"  # กำหนดพาธของฟอนต์ภาษาไทยสำหรับแสดงผล

CONFIDENCE = 0.15      # ค่าความมั่นใจขั้นต่ำ (0.0-1.0) ที่จะยอมรับผลตรวจจับจาก YOLO
OCR_INTERVAL = 15      # จะเรียกใช้ EasyOCR ทุกๆ 15 เฟรม (เพื่อลดภาระ CPU/GPU)
SAME_PLATE_COUNT = 4   # จำนวนครั้งที่ต้องอ่านค่าเดิมซ้ำกัน ถึงจะยอมรับและบันทึก (ยืนยันความเสถียร)
CLEAR_SCREEN = 45      # ถ้าไม่พบป้ายติดต่อกันเกิน 45 เฟรม ให้รีเซ็ตสถานะทั้งหมด (Timeout)
PADDING = 15           # จำนวนพิกเซลที่จะขยายกรอบภาพที่ตัด (Crop) เพื่อป้องกันขอบป้ายหลุด

# --- STATE VARIABLES (Global) ---
last_text = None       # เก็บข้อความป้ายทะเบียนที่ "บันทึกสำเร็จล่าสุด" แล้ว
last_img = None        # เก็บภาพป้ายทะเบียนที่ "บันทึกสำเร็จล่าสุด" แล้ว
pending = None         # เก็บข้อความป้ายทะเบียนที่ "กำลังรอการยืนยัน" (ยังนับไม่ครบ 4 รอบ)
empty_frames = 0       # ตัวนับจำนวนเฟรมที่ "ไม่พบ" การตรวจจับใดๆ
count = 0              # ตัวนับจำนวนครั้งที่อ่านได้ข้อความตรงกับ 'pending'
frame_counter = 0      # ตัวนับเฟรมรวมทั้งหมดตั้งแต่เริ่มโปรแกรม

# --- COLOR DEFINITIONS (BGR Format) ---
Color_Box = (203, 192, 255)    # สีม่วงอ่อน (กรอบตอนตรวจจับปกติ)
Color_Bar = (180, 158, 255)    # สีม่วงเข้ม (แถบพื้นหลังข้อความสถานะ)
Color_Text = (255, 255, 255)   # สีขาว (ข้อความสถานะ)
Color_Verify = (0, 165, 255)   # สีส้ม (กรอบตอนกำลังยืนยัน/นับ Count)

# --- SETUP AND INITIALIZATION ---
os.makedirs("saved_plates", exist_ok=True) # สร้างโฟลเดอร์สำหรับเก็บภาพป้ายทะเบียน ถ้ายังไม่มี
if not os.path.exists("parking_log.csv"): # ตรวจสอบว่าไฟล์ Log มีอยู่หรือไม่
    with open("parking_log.csv", "w", newline="", encoding="utf-8") as f: # ถ้าไม่มี ให้สร้างใหม่
        csv.writer(f).writerow(["Timestamp", "Plate", "Image"]) # เขียนหัวตารางในไฟล์ CSV

print("⏳ Loading AI...")
model = YOLO(MODEL_PATH) # โหลดโมเดล YOLO สำหรับการตรวจจับตำแหน่งเข้าหน่วยความจำ
reader = easyocr.Reader(["th"], gpu=True) # โหลดโมเดล EasyOCR ภาษาไทย (ใช้ GPU ช่วยคำนวณถ้ามี)
print("✅ Ready!")


# --- FUNCTIONS ---
def draw_text(img, text, pos, color=(255, 255, 255), size=30):
    # แปลงภาพ OpenCV (BGR) เป็น PIL Image (RGB) เพื่อให้รองรับการวาดฟอนต์ไทย
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)

    # โหลดฟอนต์ภาษาไทย (ใช้ขนาด size * 2 เพื่อความคมชัด)
    font = (
        ImageFont.truetype(FONT_PATH, size * 2)
        if os.path.exists(FONT_PATH)
        else ImageFont.load_default() # ใช้ฟอนต์พื้นฐานหากหาฟอนต์ที่กำหนดไม่เจอ
    )
    # วาดเงาข้อความ (สีเทา) ด้วยการขยับพิกัดเล็กน้อย
    draw.text((pos[0] + 2, pos[1] + 2), text, font=font, fill=(100, 100, 100))
    # วาดข้อความจริงตามตำแหน่งและสีที่กำหนด
    draw.text(pos, text, font=font, fill=color)
    # แปลงภาพกลับเป็น OpenCV Format (BGR)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)


def clean_text(text):
    # ใช้ Regex ลบอักขระที่ไม่ใช่ตัวเลข (0-9) และพยัญชนะไทย (ก-๙) ออกจากข้อความ
    return re.sub(r"[^ก-๙0-9]", "", text)


def correct_text(text):
    # ใช้ Regex ตรวจสอบ Pattern: [เลข][อักษร][อักษร][เลข]
    m = re.match(r"^(\d)(.)(.)(\d+)$", text)
    if m:
        # ดึงกลุ่มข้อมูลออกมา: p=เลขหน้า, c1=ตัวอักษร1, c2=ตัวอักษร2, s=เลขท้าย
        p, c1, c2, s = m.groups()
        # ตรวจสอบและแก้ไขตัวอักษรตัวที่ 1 ที่มักสับสนกับตัวเลข (Text Correction Logic)
        c1 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c1, c1)
        # ตรวจสอบและแก้ไขตัวอักษรตัวที่ 2 ที่มักสับสนกับตัวเลข
        c2 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c2, c2)
        # ส่งข้อความที่ประกอบใหม่แล้วกลับไป
        return f"{p}{c1}{c2}{s}"
    return text # ถ้าไม่ตรง Pattern ก็คืนค่าเดิม


def process_plate(img):
    # แปลงภาพสี (BGR) เป็นภาพขาวดำ (Grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ขยายภาพให้ใหญ่ขึ้น 2เท่า ด้วยวิธีการประมาณค่าเชิงเส้น (INTER_LINEAR)
    enhanced = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)

    # ส่งภาพที่ปรับปรุงแล้วเข้าสู่โมเดล EasyOCR
    res = reader.readtext(
        enhanced,
        detail=1, # คืนค่าเป็น Tuple (พิกัด, ข้อความ, Confidence)
        mag_ratio=2.0, # ขยายภาพภายใน EasyOCR เพิ่มเติม
        allowlist="0123456789กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ", # กำหนดชุดตัวอักษรที่อนุญาตให้อ่าน
    )

    # รวมข้อความทั้งหมดที่อ่านได้ และลบขยะด้วย clean_text
    full_text = "".join([clean_text(t) for _, t, _ in res])

    # ใช้ Regex คัดกรองและตัดข้อความส่วนเกินทิ้งให้เป็นรูปแบบป้ายทะเบียน
    m = re.match(r"^([0-9ก-๙]+\d)(?![0-9])", full_text)

    # ถ้าอ่านแล้วไม่ถึง 3 ตัวอักษร หรือไม่ตรงตาม Pattern ให้คืนค่ากลับไป None
    if not m or len(m.group(1)) < 3:
        return None

    # ส่งข้อความที่คัดกรองแล้ว ไปแก้คำผิด (correct_text) และคืนค่าสุดท้าย
    return correct_text(m.group(1))


def save_log(text, img):
    global last_text, last_img, pending, count # ต้องประกาศ global เพื่อแก้ไขค่าสถานะ
    ts = datetime.now().strftime("%Y%m%d_%H%M%S") # สร้าง Timestamp
    fname = f"saved_plates/{text}_{ts}.jpg" # กำหนดชื่อไฟล์ภาพ
    cv2.imwrite(fname, img) # บันทึกภาพที่ถูก Crop
    with open("parking_log.csv", "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow( # บันทึก Log ใน CSV
            [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), text, fname]
        )
    print(f"💾 Saved: {text}")

    # อัพเดทสถานะ
    last_text = text # ตั้งค่าข้อความที่บันทึกสำเร็จแล้ว
    last_img = img   # ตั้งค่าภาพที่บันทึกสำเร็จแล้ว
    pending = None   # ยกเลิกการรอเช็ค
    count = 0        # รีเซ็ตตัวนับ
    return fname


# --- MAIN LOOP ---
cap = cv2.VideoCapture(0) # เปิดกล้องวิดีโอ (0 คือกล้องตัวแรก)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280) # ตั้งค่าความกว้าง 1280
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) # ตั้งค่าความสูง 720

while cap.isOpened(): # วนลูปตราบเท่าที่กล้องเปิดอยู่
    ret, frame = cap.read() # อ่านภาพทีละเฟรม
    if not ret:
        break # ถ้าอ่านภาพไม่ได้ ให้หยุดลูป

    # ต้องประกาศ global ตัวแปรสถานะที่ต้องการแก้ไขภายในลูป
    global frame_counter, empty_frames, pending, count, last_text, last_img

    frame_counter += 1 # นับจำนวนเฟรมรวม
    detected = False # ตั้งค่าเริ่มต้นว่าเฟรมนี้ยังไม่พบป้าย
    h, w = frame.shape[:2] # ดึงขนาดความสูง/กว้างของเฟรม

    # 1. Object Detection (ตรวจจับวัตถุด้วย YOLO)
    for r in model(frame, conf=CONFIDENCE, verbose=False, device="mps"):
        for box in r.boxes: # วนลูปดูผลลัพธ์การตรวจจับแต่ละกล่อง
            detected = True
            x1, y1, x2, y2 = map(int, box.xyxy[0]) # ดึงพิกัด (Bounding Box)

            # ขยายขอบเขตด้วย PADDING: ขยายพิกัดกล่องเพื่อไม่ให้ภาพป้ายทะเบียนขาด
            x1, y1 = max(0, x1 - PADDING), max(0, y1 - PADDING) # ขอบบนซ้าย
            x2, y2 = min(w, x2 + PADDING), min(h, y2 + PADDING) # ขอบล่างขวา

            # สีกรอบ: ตรวจสอบสถานะ count
            color = Color_Verify if count > 0 else Color_Box # ถ้ากำลังนับ (count > 0) ใช้สีส้ม มิฉะนั้นใช้สีม่วงอ่อน
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3) # วาดกรอบสี่เหลี่ยม

            # 2. OCR Logic (ตรรกะการอ่านข้อความ)
            # เงื่อนไข: ถ้ายังไม่มีข้อความบันทึกสำเร็จ (not last_text) และถึงรอบอ่าน (frame_counter % OCR_INTERVAL == 0)
            if not last_text and frame_counter % OCR_INTERVAL == 0:

                # Crop แค่ป้ายทะเบียนไป Process: ส่งภาพป้ายทะเบียนที่ตัดแล้วไปประมวลผล OCR
                plate_text = process_plate(frame[y1:y2, x1:x2])
                if plate_text: # ถ้าอ่านข้อความออกมาได้สำเร็จ
                    # ถ้าเลขตรงกันกับที่รอเช็คอยู่ จะเพิ่มCount +1
                    if plate_text == pending:
                        count += 1 # ตรงกัน: นับเพิ่ม
                    else:
                        # ถ้าไม่ตรง (หรือเป็นค่าใหม่) ให้ตั้งค่า 'pending' ใหม่และเริ่มนับ Count = 1
                        pending = plate_text
                        count = 1

                    print(f"Checking: {plate_text} {count}/{SAME_PLATE_COUNT})")
                    # ถ้าจำนวน Count ถึงเกณฑ์ (SAME_PLATE_COUNT = 4) ให้บันทึก
                    if count >= SAME_PLATE_COUNT:
                        save_log(plate_text, frame[y1:y2, x1:x2]) # บันทึก Log และรีเซ็ตสถานะ

    # 3. Reset Logic (ตรรกะรีเซ็ตเมื่อรถออกไปแล้ว)
    # ถ้าไม่เจอรถ (not detected) และไม่มีค่าที่กำลังรอเช็ค (not pending)
    if not detected and not pending:
        empty_frames += 1 # นับจำนวนเฟรมที่ว่างเปล่าเพิ่ม
    else:
        empty_frames = 0 # ถ้าเจอรถหรือมีค่า pending ให้รีเซ็ตตัวนับเฟรมว่าง

    # ถ้าเฟรมว่างมากกว่า 45 เฟรม (หมดเวลา Time-out)
    if empty_frames > CLEAR_SCREEN:
        if last_text:
            print("🚗 Clear") # พิมพ์สถานะว่าเคลียร์แล้ว
        # รีเซ็ตสถานะทั้งหมด (พร้อมรับรถคันใหม่)
        last_text = None
        last_img = None
        pending = None
        count = 0
        empty_frames = 0

    # 4. UI (User Interface)
    # Message Show All: กำหนดข้อความสถานะที่จะแสดงบนหน้าจอ
     msg = (
        f"บันทึกสำเร็จ: {last_text}"  # ข้อความกรณีที่ 1: บันทึกแล้ว
        if last_text
        else (
            f"กำลังยืนยัน {count}/{SAME_PLATE_COUNT}: {pending}"  # ข้อความกรณีที่ 2: กำลังยืนยัน
            if count > 0
            else (f"กำลังตรวจจับป้าย..." if detected else f"กำลังรอรถ...")  # ข้อความกรณีที่ 3
        )
    )

    cv2.rectangle(frame, (0, 0), (1280, 100), Color_Bar, -1)  # วาดแถบพื้นหลังด้านบน
    frame = draw_text(frame, msg, (10, 10), Color_Text, 30)  # วาดข้อความสถานะ

    # แสดงรูป Thumbnail มุมขวาบน (ถ้ามี)
    if last_img is not None:  # ตรวจสอบว่ามีรูปล่าสุดไหม
        try:  # ลองทำคำสั่งในนี้ (Try block)
            # คำนวณขนาดภาพย่อ
            thumbnail_heigh, thumbnail_width = 80, int(
                80 * (last_img.shape[1] / last_img.shape[0])
            )
            # วางภาพย่อลงบนเฟรมหลัก
            frame[
                10 : 10 + thumbnail_heigh, 1280 - thumbnail_width - 20 : 1280 - 20
            ] = cv2.resize(last_img, (thumbnail_width, thumbnail_heigh))
        except:  # ถ้าเกิด error (เช่น ขนาดภาพผิด)
            pass  # ข้ามไป ไม่ต้องทำอะไร

    cv2.imshow("Smart Parking", frame)  # แสดงภาพหน้าต่างโปรแกรม
    if cv2.waitKey(1) == ord("q"):  # รอรับปุ่มกด 1ms, ถ้าเป็นปุ่ม q
        break  # สั่งหยุดลูป

cap.release()  # คืนค่าการเชื่อมต่อกล้อง
cv2.destroyAllWindows()  # ปิดหน้าต่างโปรแกรมทั้งหมด

In [1]:
import csv
import os
import re
import threading  # <--- เพิ่ม threading
import time
from datetime import datetime

import cv2
import easyocr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO

# ส่วนของเสียง
from gtts import gTTS
# หาก playsound มีปัญหาบน Mac ให้ใช้ os.system("afplay temp.mp3") แทน

MODEL_PATH = "./Train/thai_plate_v1_fast4/weights/best.pt"
FONT_PATH = "/System/Library/Fonts/Supplemental/Ayuthaya.ttf"

CONFIDENCE = 0.15
OCR_INTERVAL = 15
SAME_PLATE_COUNT = 4
CLEAR_SCREEN = 45
PADDING = 15

last_text = None
last_img = None
pending = None
empty_frames = 0
count = 0
frame_counter = 0

Color_Box = (203, 192, 255)
Color_Bar = (180, 158, 255)
Color_Text = (255, 255, 255)
Color_Verify = (0, 165, 255)


os.makedirs("saved_plates", exist_ok=True)
if not os.path.exists("parking_log.csv"):
    with open("parking_log.csv", "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["Timestamp", "Plate", "Image"])

print("⏳ Loading AI...")
model = YOLO(MODEL_PATH)
reader = easyocr.Reader(["th"], gpu=True)
print("✅ Ready!")


def draw_text(img, text, pos, color=(255, 255, 255), size=30):
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    font = (
        ImageFont.truetype(FONT_PATH, size * 2)
        if os.path.exists(FONT_PATH)
        else ImageFont.load_default()
    )
    draw.text((pos[0] + 2, pos[1] + 2), text, font=font, fill=(100, 100, 100))
    draw.text(pos, text, font=font, fill=color)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)


def clean_text(text):
    return re.sub(r"[^ก-๙0-9]", "", text)


def correct_text(text):
    m = re.match(r"^(\d)(.)(.)(\d+)$", text)
    if m:
        p, c1, c2, s = m.groups()
        c1 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c1, c1)
        c2 = {"1": "ม", "5": "ร", "6": "ธ", "8": "ย", "4": "ภ", "0": "จ"}.get(c2, c2)
        return f"{p}{c1}{c2}{s}"
    return text


def process_plate(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    enhanced = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    res = reader.readtext(
        enhanced,
        detail=1,
        mag_ratio=2.0,
        allowlist="0123456789กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ",
    )
    full_text = "".join([clean_text(t) for _, t, _ in res])
    m = re.match(r"^([0-9ก-๙]+\d)(?![0-9])", full_text)
    if not m or len(m.group(1)) < 3:
        return None
    return correct_text(m.group(1))


# --- ฟังก์ชันสำหรับพูด (ทำงานแยก Thread) ---
def speak_thread(text):
    try:
        # แยกตัวอักษรให้อ่านง่ายขึ้น (เช่น 1กข1234 -> 1 กข 1 2 3 4)
        # หรือจะส่ง text ไปตรงๆ เลยก็ได้ Google ฉลาดพอสมควร
        spoken_text = f"ยินดีต้อนรับทะเบียน {text}"

        tts = gTTS(text=spoken_text, lang='th')
        filename = f"temp_speech_{int(time.time())}.mp3"
        tts.save(filename)

        # สำหรับ macOS ใช้ afplay จะเสถียรกว่า playsound
        os.system(f"afplay {filename}")
        # ถ้าเป็น Windows ใช้: os.system(f"start {filename}")

        os.remove(filename) # ลบไฟล์ทิ้งเมื่อพูดจบ
    except Exception as e:
        print(f"🔊 Speech Error: {e}")

def announce_plate(text):
    # สั่งให้พูดใน Thread ใหม่ เพื่อไม่ให้กล้องค้าง
    threading.Thread(target=speak_thread, args=(text,), daemon=True).start()
# ----------------------------------------


def save_log(text, img):
    global last_text, last_img, pending, count
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"saved_plates/{text}_{ts}.jpg"
    cv2.imwrite(fname, img)
    with open("parking_log.csv", "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(
            [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), text, fname]
        )
    print(f"💾 Saved: {text}")

    # --- เรียกใช้เสียง ---
    announce_plate(text)
    # ------------------

    last_text = text
    last_img = img
    pending = None
    count = 0
    return fname


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    detected = False
    h, w = frame.shape[:2]

    # ใช้ stream=True เพื่อลด memory overhead (optional)
    results = model(frame, conf=CONFIDENCE, verbose=False, device="mps")

    for r in results:
        for box in r.boxes:
            detected = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            x1, y1 = max(0, x1 - PADDING), max(0, y1 - PADDING)
            x2, y2 = min(w, x2 + PADDING), min(h, y2 + PADDING)

            color = Color_Verify if count > 0 else Color_Box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

            if not last_text and frame_counter % OCR_INTERVAL == 0:
                plate_text = process_plate(frame[y1:y2, x1:x2])
                if plate_text:
                    if plate_text == pending:
                        count += 1
                    else:
                        pending = plate_text
                        count = 1

                    print(f"Checking: {plate_text} {count}/{SAME_PLATE_COUNT})")

                    if count >= SAME_PLATE_COUNT:
                        save_log(plate_text, frame[y1:y2, x1:x2])

    if not detected and not pending:
        empty_frames += 1
    else:
        empty_frames = 0

    if empty_frames > CLEAR_SCREEN:
        if last_text:
            print("🚗 Clear")
        last_text = None
        last_img = None
        pending = None
        count = 0
        empty_frames = 0

    msg = (
        f"บันทึกสำเร็จ: {last_text}"
        if last_text
        else (
            f"กำลังยืนยัน {count}/{SAME_PLATE_COUNT}: {pending}"
            if count > 0
            else (f"กำลังตรวจจับป้าย..." if detected else f"กำลังรอรถ...")
        )
    )

    cv2.rectangle(frame, (0, 0), (1280, 100), Color_Bar, -1)
    frame = draw_text(frame, msg, (10, 10), Color_Text, 30)

    if last_img is not None:
        try:
            thumbnail_heigh, thumbnail_width = 80, int(
                80 * (last_img.shape[1] / last_img.shape[0])
            )
            frame[
                10 : 10 + thumbnail_heigh, 1280 - thumbnail_width - 20 : 1280 - 20
            ] = cv2.resize(last_img, (thumbnail_width, thumbnail_heigh))
        except:
            pass

    cv2.imshow("Smart Parking", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

⏳ Loading AI...
✅ Ready!


[ WARN:0@4.941] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Checking: 1กค771 1/4)
Checking: 1กค771 2/4)
Checking: 1กค771 3/4)
Checking: 1กค771 4/4)
💾 Saved: 1กค771
🚗 Clear
